In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.model_selection import train_test_split
from zipfile import ZipFile
from glob import glob
from xgboost import XGBRegressor

In [2]:
#read zip files, print directories, extract all files
with ZipFile('data.zip', 'r') as zip:
    zip.printdir()
    zip.extractall() 

File Name                                             Modified             Size
competition_data/                              2015-06-29 21:06:12            0
competition_data/bill_of_materials.csv         2015-06-24 16:21:48      1326876
competition_data/comp_adaptor.csv              2015-06-24 16:21:48         2544
competition_data/comp_boss.csv                 2015-06-24 16:21:48        10695
competition_data/comp_elbow.csv                2015-06-24 16:21:48        12830
competition_data/comp_float.csv                2015-06-24 16:21:48          703
competition_data/comp_hfl.csv                  2015-06-24 16:21:48          438
competition_data/comp_nut.csv                  2015-06-24 16:21:48         3324
competition_data/comp_other.csv                2015-06-29 21:05:42        20941
competition_data/comp_sleeve.csv               2015-06-24 16:21:48         2565
competition_data/comp_straight.csv             2015-06-24 16:21:48        19721
competition_data/comp_tee.csv           

In [3]:
sample_sub= open('sample_submission.csv', 'r')

In [4]:
sample_sub= pd.read_csv(sample_sub)

In [5]:
#setting display option so i can see all columns
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [6]:
#Loaded train, test, tube, and bill data into pandas dataframes
test_data= pd.read_csv('competition_data/test_set.csv')
train_data= pd.read_csv('competition_data/train_set.csv')
tube_data= pd.read_csv('competition_data/tube.csv')
bill_data= pd.read_csv('competition_data/bill_of_materials.csv')
type_comp_data= pd.read_csv('competition_data/type_component.csv')
type_connect_data= pd.read_csv('competition_data/type_connection.csv')
type_end_form= pd.read_csv('competition_data/type_end_form.csv')

In [14]:
type_comp_data

,component_type_id,name
0,CP-001,4-bolt Tig Straight
1,CP-002,4-bolt MJ Straight
2,CP-003,4-bolt Braze/Weld Straight
3,CP-004,2-bolt Braze/Weld Straight
4,CP-005,2-bolt MJ Straight
5,CP-006,Braze/Weld Flange Head
6,CP-007,MJ Flange Head
7,CP-008,4-bolt Braze/Weld Elbow
8,CP-009,4-bolt Tig Elbow
9,CP-010,2-bolt Braze/Weld Elbow


In [11]:
type_end_form.head()

,end_form_id,name
0,A-001,Male (Stud)
1,A-002,Male (Swivel)
2,A-003,Braze-Weld Boss
3,A-004,Braze-Weld Socket
4,A-005,Swivel Nut


In [10]:
type_connect_data.head()

,connection_type_id,name
0,B-001,37 deg Flare-SAE J514
1,B-002,ORFS-SAE J1453
2,B-003,Hi-Duty
3,B-004,NPTF-SAE J476/J514
4,B-005,SAE STOR-SAE J1926


29

In [12]:
#load in all components data into pandas dataframes
components= pd.read_csv('competition_data\components.csv')
adaptors= pd.read_csv('competition_data\comp_adaptor.csv')
boss= pd.read_csv('competition_data\comp_boss.csv')
elbow= pd.read_csv('competition_data\comp_elbow.csv')
comp_float= pd.read_csv('competition_data\comp_float.csv')
hfl= pd.read_csv('competition_data\comp_hfl.csv')
nut= pd.read_csv('competition_data\comp_nut.csv')
other= pd.read_csv('competition_data\comp_other.csv')
sleeve= pd.read_csv('competition_data\comp_sleeve.csv')
straight= pd.read_csv('competition_data\comp_straight.csv')
tee= pd.read_csv('competition_data\comp_tee.csv')
threaded= pd.read_csv('competition_data\comp_threaded.csv')

In [29]:
components_with_type= pd.merge(components, type_comp_data, how='inner', on='component_type_id')

In [30]:
components_with_type

,component_id,name_x,component_type_id,name_y
0,C-0001,SLEEVE,CP-024,Sleeves
1,C-0002,SLEEVE,CP-024,Sleeves
2,C-0003,SLEEVE-FLARED,CP-024,Sleeves
3,C-0048,SLEEVE-BRAZE,CP-024,Sleeves
4,C-0049,SLEEVE-ORFS,CP-024,Sleeves
5,C-0050,SLEEVE-ORFS,CP-024,Sleeves
6,C-0051,SLEEVE-ORFS,CP-024,Sleeves
7,C-0052,SLEEVE-ORFS,CP-024,Sleeves
8,C-0053,SLEEVE-ORFS,CP-024,Sleeves
9,C-0058,SLEEVE-90DEG,CP-024,Sleeves


In [ ]:
#merging all components data into a final components dataframe
adapt_merge= pd.merge(components, adaptors, how= 'outer', on='component_id')
boss_merge= pd.merge(adapt_merge, boss, how= 'outer', on= 'component_id')
elbow_merge= pd.merge(boss_merge, elbow, how= 'outer', on= 'component_id')
float_merge= pd.merge(elbow_merge, comp_float, how= 'outer', on= 'component_id')
hfl_merge= pd.merge(float_merge, hfl, how= 'outer', on= 'component_id')
nut_merge= pd.merge(hfl_merge, nut, how= 'outer', on= 'component_id')
other_merge= pd.merge(nut_merge, other, how= 'outer', on= 'component_id')
sleeve_merge= pd.merge(other_merge, sleeve, how= 'outer', on= 'component_id')
straight_merge= pd.merge(sleeve_merge, straight, how= 'outer', on= 'component_id')
tee_merge= pd.merge(straight_merge, tee, how= 'outer', on= 'component_id')
final_merge= pd.merge(tee_merge, threaded, how= 'outer', on= 'component_id')

In [ ]:
components.shape, final_merge.shape

In [ ]:
test_set= test_data.copy()
train_set= train_data.copy()

In [ ]:
#getting rid of duplicate columns
components_data = final_merge.loc[:,~final_merge.columns.duplicated()]

In [ ]:
components_data.shape

In [ ]:
# columns= ['tube_assembly_id', 'material_id', 'diameter', 'wall', 'length', 'bend_radius']
tube_merge= tube_data.copy()

In [ ]:
train_set.shape,test_set.shape,  tube_merge.shape

In [ ]:
train_set= pd.merge(train_set, tube_merge)
test_set= pd.merge(test_set, tube_merge)

In [ ]:
train_set.shape, test_set.shape, bill_data.shape

In [ ]:
train_set= pd.merge(train_set, bill_data)
test_set= pd.merge(test_set, bill_data)

In [ ]:
train_set.shape, test_set.shape

In [ ]:
train_component_1= train_set['component_id_1'].unique()
train_component_2= train_set['component_id_2'].unique()
train_component_3= train_set['component_id_3'].unique()
train_component_4= train_set['component_id_4'].unique()
train_component_5= train_set['component_id_5'].unique()
train_component_6= train_set['component_id_6'].unique()
train_component_7= train_set['component_id_7'].unique()
train_component_8= train_set['component_id_8'].unique()

In [ ]:
test_component_1= test_set['component_id_1'].unique()
test_component_2= test_set['component_id_2'].unique()
test_component_3= test_set['component_id_3'].unique()
test_component_4= test_set['component_id_4'].unique()
test_component_5= test_set['component_id_5'].unique()
test_component_6= test_set['component_id_6'].unique()
test_component_7= test_set['component_id_7'].unique()
test_component_8= test_set['component_id_8'].unique()

In [ ]:
train_comp_one_data= components_data[components_data['component_id'].isin(train_component_1)]
train_comp_one_data= train_comp_one_data.rename(columns= {'component_id': 'component_id_1'})
train_comp_two_data= components_data[components_data['component_id'].isin(train_component_2)]
train_comp_two_data= train_comp_two_data.rename(columns={'component_id': 'component_id_2'})
train_comp_three_data= components_data[components_data['component_id'].isin(train_component_3)]
train_comp_three_data= train_comp_three_data.rename(columns={'component_id': 'component_id_3'})
train_comp_four_data= components_data[components_data['component_id'].isin(train_component_4)]
train_comp_four_data= train_comp_four_data.rename(columns={'component_id': 'component_id_4'})
train_comp_five_data= components_data[components_data['component_id'].isin(train_component_5)]
train_comp_five_data= train_comp_five_data.rename(columns={'component_id': 'component_id_5'})
train_comp_six_data= components_data[components_data['component_id'].isin(train_component_6)]
train_comp_six_data= train_comp_six_data.rename(columns={'component_id': 'component_id_6'})
train_comp_seven_data= components_data[components_data['component_id'].isin(train_component_7)]
train_comp_seven_data= train_comp_seven_data.rename(columns={'component_id': 'component_id_7'})
train_comp_eight_data= components_data[components_data['component_id'].isin(train_component_8)]
train_comp_eight_data= train_comp_eight_data.rename(columns={'component_id': 'component_id_8'})

In [ ]:
test_comp_one_data= components_data[components_data['component_id'].isin(test_component_1)]
test_comp_one_data= test_comp_one_data.rename(columns= {'component_id': 'component_id_1'})
test_comp_two_data= components_data[components_data['component_id'].isin(test_component_2)]
test_comp_two_data= test_comp_two_data.rename(columns={'component_id': 'component_id_2'})
test_comp_three_data= components_data[components_data['component_id'].isin(test_component_3)]
test_comp_three_data= test_comp_three_data.rename(columns={'component_id': 'component_id_3'})
test_comp_four_data= components_data[components_data['component_id'].isin(test_component_4)]
test_comp_four_data= test_comp_four_data.rename(columns={'component_id': 'component_id_4'})
test_comp_five_data= components_data[components_data['component_id'].isin(test_component_5)]
test_comp_five_data= test_comp_five_data.rename(columns={'component_id': 'component_id_5'})
test_comp_six_data= components_data[components_data['component_id'].isin(test_component_6)]
test_comp_six_data= test_comp_six_data.rename(columns={'component_id': 'component_id_6'})
test_comp_seven_data= components_data[components_data['component_id'].isin(test_component_7)]
test_comp_seven_data= test_comp_seven_data.rename(columns={'component_id': 'component_id_7'})
test_comp_eight_data= components_data[components_data['component_id'].isin(test_component_8)]
test_comp_eight_data= test_comp_eight_data.rename(columns={'component_id': 'component_id_8'})

In [ ]:
train_set= pd.merge(train_set, train_comp_one_data, how='outer', on='component_id_1')
test_set= pd.merge(test_set, test_comp_one_data, how='outer', on='component_id_1')

In [ ]:
train_set.shape, test_set.shape

In [ ]:
train_set= pd.merge(train_set, train_comp_two_data, how='outer', on='component_id_2')
test_set= pd.merge(test_set, test_comp_two_data, how='outer', on='component_id_2')

In [ ]:
train_set.shape, test_set.shape

In [ ]:
train_set= pd.merge(train_set, train_comp_three_data, how='outer', on='component_id_3')
test_set= pd.merge(test_set, test_comp_three_data, how='outer', on='component_id_3')

In [ ]:
train_set.shape, test_set.shape

In [ ]:
train_set= pd.merge(train_set, train_comp_four_data, how='outer', on='component_id_4')
test_set= pd.merge(test_set, test_comp_four_data, how='outer', on='component_id_4')

In [ ]:
train_set.shape, test_set.shape

In [ ]:
train_set= pd.merge(train_set, train_comp_five_data, how='outer', on='component_id_5')
test_set= pd.merge(test_set, test_comp_five_data, how='outer', on='component_id_5')

In [ ]:
train_set.shape, test_set.shape

In [ ]:
train_set= pd.merge(train_set, train_comp_six_data, how='outer', on='component_id_6')
test_set= pd.merge(test_set, test_comp_six_data, how='outer', on='component_id_6')

In [ ]:
train_set.shape, test_set.shape

In [ ]:
train_set= pd.merge(train_set, train_comp_seven_data, how='outer', on='component_id_7')
test_set= pd.merge(test_set, test_comp_seven_data, how='outer', on='component_id_7')

In [ ]:
train_set.shape, test_set.shape

In [ ]:
train_set= pd.merge(train_set, train_comp_eight_data, how='outer', on='component_id_8')
test_set= pd.merge(test_set, test_comp_eight_data, how='outer', on='component_id_8')

In [ ]:
train_set.shape, test_set.shape

In [ ]:
train_set= train_set.loc[:,~train_set.columns.duplicated()]
test_set= test_set.loc[:,~test_set.columns.duplicated()]

In [ ]:
train_set.shape, test_set.shape

In [ ]:
train_set= train_set.replace([np.inf, -np.inf], np.nan)
test_set= test_set.replace([np.inf, -np.inf], np.nan)

In [ ]:
train_set= train_set.fillna(0)
test_set= test_set.fillna(0)

In [ ]:
#grabbed the unique tube assembly ids for train and test data
train_assembly_unique= train_set['tube_assembly_id'].unique()
test_assembly_unique= test_set['tube_assembly_id']. unique()

In [ ]:
#split the unique assembies into train and validate groups
train_assemblies0, test_assemblies0= train_test_split(train_assembly_unique, train_size= 0.80, test_size=0.2, random_state= 42)

In [ ]:
#looked at shape of both train and val assemblies
train_assemblies0.shape, test_assemblies0.shape

In [ ]:
train_assemblies1, test_assemblies1= train_test_split(train_assemblies0, train_size= 0.80, test_size= 0.2, random_state=42)

In [ ]:
#created train an val datasets by grabbing only data associated with the unique assemblies for that dataset
train0= train_set[train_set['tube_assembly_id'].isin(train_assemblies0)]
val0= train_set[train_set['tube_assembly_id'].isin(test_assemblies0)]

In [ ]:
train1= train_set[train_set['tube_assembly_id'].isin(train_assemblies1)]
val1= train_set[train_set['tube_assembly_id'].isin(test_assemblies1)]

In [ ]:
#checked the shape of train and validate
train0.shape, val0.shape

In [ ]:
#created a wrangle function
def wrangle(df):
    df= df.copy()
    df['quote_date']= pd.to_datetime(df['quote_date'], infer_datetime_format= True)
    df['quote_date_year'] = df['quote_date'].dt.year
    df['quote_date_month'] = df['quote_date'].dt.month
    df= df.drop(columns=['quote_date'])
    return df

In [ ]:
train0= wrangle(train0)
val0= wrangle(val0)
# train1=wrangle(train1)
# val1= wrangle(val1)
test= wrangle(test_set)

In [ ]:
train0.shape, val0.shape, train1.shape, val1.shape, test.shape

In [ ]:
#log target
train0_target= np.log1p(train0['cost'])
# train1_target= np.log1p(train1['cost'])
val0_target= np.log1p(val0['cost'])
# val1_target= np.log1p(val1['cost'])

#drop target and tube id
train0_features= train0.drop(columns=['cost', 'tube_assembly_id'])
val0_features= val0.drop(columns=['cost', 'tube_assembly_id'])
# train1_features= train1.drop(columns=['cost', 'tube_assembly_id'])
# val1_features= val1.drop(columns=['cost', 'tube_assembly_id'])
test_ids= test['id']
test= test.drop(columns=['id','tube_assembly_id'])

In [ ]:
test.shape, sample_sub.shape

In [ ]:
#created a pipeline with an ordinal encoder and randomforestregressor
pipeline= make_pipeline(ce.OrdinalEncoder(),
                       RandomForestRegressor(n_estimators=1000, random_state=42, n_jobs=-1))

In [ ]:
pipeline.fit(train0_features, train0_target)

In [ ]:
#fitted training features and target to pipeline
pipeline.fit(train1_features, train1_target)

In [ ]:
#made predictions on validations features
# y_pred1= pipeline.predict(val1_features)
#extra validation
y_pred0= pipeline.predict(val0_features)


In [ ]:
#root mean squared error with log target for validation and extra validation
# np.sqrt(mean_squared_error(val1_target, y_pred1)), 
np.sqrt(mean_squared_error(val0_target, y_pred0))

In [ ]:
#R squared scores 
# r2_score(val1_target, y_pred1), 
r2_score(val0_target, y_pred0)

In [ ]:
y_pred= pipeline.predict(test)

In [ ]:
y_pred= np.expm1(y_pred)

In [ ]:
sample_sub['cost']= y_pred


In [ ]:
sample_sub.to_csv('sample_submission.csv', index=False)

In [ ]:
encoder= ce.OrdinalEncoder()
# train1_encoded= encoder.fit_transform(train1_features)
# val1_encoded= encoder.transform(val1_features)
train0_encoded= encoder.fit_transform(train0_features)
val0_encoded= encoder.transform(val0_features)
test_encoded= encoder.transform(test)

In [ ]:
# eval_set = [(train1_encoded, train1_target), 
#             (val1_encoded, val1_target)]
model= XGBRegressor(n_estimators= 1000,eval_metric='rmse', early_stopping_rounds=10, random_state=42, n_jobs=-1)

In [ ]:
model.fit(train0_encoded, train0_target)

In [ ]:
# y_pred1= model.predict(val1_encoded)
y_pred0= model.predict(val0_encoded)

In [ ]:
# np.sqrt(mean_squared_error(val1_target, y_pred1)), 
np.sqrt(mean_squared_error(val0_target, y_pred0))

In [ ]:
# r2_score(val1_target, y_pred1), 
r2_score(val0_target, y_pred0)


In [ ]:
columns= ['end_form_id_3_x','connection_type_id_3_x', 'mj_plug_class_code_y_y']
test_encoded[columns]= test_encoded[columns].replace(to_replace= ['A-001', 'B-002', 'MJ-005'], value=0)

In [ ]:
sub_pred= model.predict(test_encoded)

In [ ]:
sub_pred.shape

In [ ]:
sub_pred= np.expm1(sub_pred)

In [ ]:
sample_sub['cost']= sub_pred

In [ ]:
sample_sub.to_csv('sample_submission.csv', index=False)

In [ ]:
sample_sub.head()

In [ ]:
test_set.head()